This notebook demonstrate the process of fine tuning a GPT model using the OpenAI API endpoint


### Install packages

In [ ]:
#install required packages for Python environment
!pip install openai

In [ ]:
#import required packages
import os
from openai import OpenAI
import json
import pandas as pd

To be able to use OpenAI you will need to connect to OpenAI via a API key. You can find your Secret API key on the [API key page](https://platform.openai.com/api-keys).

Check out our [Best Practices for API Key Safety to learn how you can keep your API key protected](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety).

In [ ]:
#Using your API key, start a new OpenAI
session with your credentials

os.environ['OPENAI_API_KEY'] = 'paste API key here'
client=OpenAI()


### Import your training data
You can import your training data either directly via Github (If you are set up there) or by using your google drive if you are using Google Colab.

In [ ]:
#Mount Google Drive (Only if you want to work via Google Colab )

from google.colab import drive
drive.mount('/content/drive')


Import data for fine tuning. The data must be in JSON Lines format and must contain at least 10 examples in OpenAI 'message' structure.
OpenAI documentation on formatting can be found here: https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset


In [ ]:

file_path='/content/drive/MyDrive/training_data.json'

client.files.create(
  file=open(file_path, 'rb'),
  purpose="fine-tune"
)


#This code block will return a file name as 'training_file'. Take note of this for the next block.
#Alternatively, store the file name as a new variable.

This code block will return a file name as 'training_file'. Take note of this for the next block.

Alternatively, store the file name as a new variable.

### Fine Tuning

Create a new fine tuning job using the file created in the last step.

Additional parameters you can include are outlined in OpenAI's documentation: https://platform.openai.com/docs/api-reference/fine-tuning

In [ ]:

client.fine_tuning.jobs.create(
  training_file='paste the name of training_file from previous step here',
  model="gpt-3.5-turbo" #Select specific GPT model for fine tuning. See the OpenAI documentation for more information, including costs.
)

As in the step above, this code block will return the name of fine tuning job. Take note of this for the next block.

In [ ]:
#To check the status of a fine tuning job:
client.fine_tuning.jobs.retrieve('paste fine tuning job from previous step here')


Ensure the fine tuning job is running and no errors have arisen.

When the fine tuning has finished, running this line of code will show that the output of the fine_tuned_model argument has been populated with a unique name (which will start with 'ft: '). As in the previous steps, take note of the model's name.

Depending on the size of the training file, fine tuning can take some time. As long as the status shows the job is 'running', you can safely assume no errors have arisen.


In [ ]:
#To cancel a fine-tuning job:
#client.fine_tuning.jobs.cancel('paste fine tuning job from previous step here')

#To delete a fine-tuned model:
#client.models.delete('paste fine tuning job from previous step here')

In [ ]:
#Define the system message, which sets an overall identity or 'personality' for the model, and a question for testing the model:

system_msg: 'You are MunroBot, and you answer questions about the mountains in Scotland in the voice of 19th-century moutaineer Hugh Munro.'
input_msg: 'MunroBot, how many peaks are in the Cuillin Ridge?'



In [ ]:
#Test the model using the system message and input question:

completion = client.chat.completions.create(
  model='name of fine-tuned model',
  messages=[
    {'role': 'system', 'content': system_msg},
    {"role": "user", "content": input_msg}
  ]
)

#Print the model's response:
print(completion.choices[0].message)

In [ ]:
#Depending on your use case, you can plug the fine-tuned model into a basic chatbot interface with the gradio package:

import gradio as gr

def chatbot(prompt):
  completion = client.chat.completions.create(
      model='name of fine-tuned model',
      temperature=0.6,
      messages=[
    {'role': 'system', 'content': system_msg},
    {"role": "user", "content": input_msg}
  ]
)
  return (completion.choices[0].message.content)

iface = gr.Interface(fn=chatbot, inputs='text', outputs='text')
iface.launch(share=True)